In [ ]:
# just a demo:
# the following redirects future cells through this loop.
# This should make it possible to put wrapper functionality
# around cells to keep the text inside the cells clean.
# (currently disabled with the false &&)

false && while sleep 1; do
echo -n "$PS1"
read ln
eval "$ln"
echo "It worked! " # If that last space is removed, it does not work :-(
done
echo $$ # for debugging

In [ ]:
# (1) optional reset of Wakame-vdc
# Comment out the following lines and run cell to remove all
# added images and instances.

# reset-wakame.sh


In [ ]:
# (2) Create Centos instance using mussel

cat <<EOS >vifs.json
{
 "eth0":{"network":"nw-demo1","security_groups":"sg-cicddemo"}
}
EOS

mussel instance create --cpu-cores 2 --hypervisor openvz \
    --image-id wmi-centos1d64 --memory-size 2048 \
    --ssh-key-id ssh-cicddemo --vifs vifs.json --display-name centos

In [ ]:
# (3) Wait for instance's ssh port to become active
: ${IP:=10.0.2.100}
while [[ "$(echo | nc -w 1 "$IP" 22)" != *SSH* ]]; do
  sleep 1
done
sleep 30
echo "SSH is active at $IP, port 22"

## repoファイルのインストール

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 curl -fSkL http://pkg.jenkins-ci.org/redhat/jenkins.repo -o /etc/yum.repos.d/jenkins.repo


## 公開鍵のインポート

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 rpm --import http://pkg.jenkins-ci.org/redhat/jenkins-ci.org.key

## java実行環境のインストール

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 yum install -y java-1.7.0-openjdk

## jenkins coreのインストール

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 yum install -y jenkins

## 関連するビルドツールのインストール

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 yum install -y \
    git \
    iputils nc \
    qemu-kvm qemu-img \
    parted kpartx \
    rpm-build automake createrepo \
    openssl-devel zlib-devel readline-devel \
    gcc


## sudo設定

### sudo権限の付与


In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 "echo 'jenkins ALL=(ALL) NOPASSWD: ALL' >>/etc/sudoers"
ssh -qi ../mykeypair root@10.0.2.100 grep jenkins /etc/sudoers

### tty設定の変更

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 'sed -i "s/^\(^Defaults\s*requiretty\).*/# \1/" /etc/sudoers'

### Loosen Java Security so Jenkins Plugins will work

In [ ]:
# see https://issues.jenkins-ci.org/browse/JENKINS-31089
javadir=/usr/lib/jvm/java-1.7.0-openjdk-1.7.0.95.x86_64/
secfile=jre/lib/security/java.security
ssh -qi ../mykeypair root@10.0.2.100 \
   "sudo sed -i 's/, MD5, RSA keySize < 1024//' $javadir/$secfile"


## jenkinsの起動


In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 service jenkins start